<a href="https://colab.research.google.com/github/Ariechovi/kaggle_results/blob/main/kaggle_%E6%95%B0%E5%AD%97%E8%AF%86%E5%88%AB%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 数据读入阶段

In [27]:
from google.colab import drive
drive.mount('/content/drive')

train_path = '/content/drive/MyDrive/My/kaggle_data/digit-recognizer/train.csv'
test_path = '/content/drive/MyDrive/My/kaggle_data/digit-recognizer/test.csv'
sample_path = '/content/drive/MyDrive/My/kaggle_data/digit-recognizer/sample_submission.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:

import pandas as pd
import torch
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader,Dataset
from torch import nn
from sklearn.model_selection import train_test_split

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
sample_df = pd.read_csv(sample_path)

# 数据准备阶段
**目标--把这些数据转成CNN能直接吃的数据格式**
- 模型的输入需要什么形状？
- 训练集如何存储的？
## CNN存储
- 输入的格式是`[batch_size, channel, height, width]`
- 现在的数据格式是
  - 第一列是类别
  - 剩下的是像素值
## 处理目标
- X : [batch_size,1,28,28]，像素值归一化到[0,1]区间
- Y：[batch_size]，标签

In [29]:
X_train=train_df.drop(columns=['label']).values
y_train=train_df['label'].values

In [30]:
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=42)
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

(33600, 784) (8400, 784)
(33600,) (8400,)


In [31]:
X_train=X_train/255.0#输入归一化
X_train=X_train.reshape(-1,1,28,28)#转换输入形状
X_val=X_val/255.0
X_val=X_val.reshape(-1,1,28,28)

In [32]:
X_train_tensor=torch.tensor(X_train,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train,dtype=torch.long)#转换成向量
X_val_tensor=torch.tensor(X_val,dtype=torch.float32)
y_val_tensor=torch.tensor(y_val,dtype=torch.long)

In [33]:
train_dataset=TensorDataset(X_train_tensor,y_train_tensor)
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)
val_dataset=TensorDataset(X_val_tensor,y_val_tensor)
val_loader=DataLoader(val_dataset,batch_size=64,shuffle=False)

In [34]:
test_images=test_df.values
test_images=test_images.reshape(-1,1,28,28) /255.0
test_images=torch.tensor(test_images,dtype=torch.float32)

class TestDataset(Dataset):
    def __init__(self, images):
        self.images = images
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        return self.images[idx]

test_dataset = TestDataset(test_images)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [35]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN,self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(32,64,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128,10)
        )
    def forward(self,x):
        x=self.conv_layers(x)
        x=self.fc_layers(x)
        return x

In [36]:
# 判断是否有GPU，若有GPU就使用GPU，否则使用CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [37]:
model=SimpleCNN().to(device)

In [38]:
loss_fn=nn.CrossEntropyLoss()

In [39]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [40]:
num_epochs=11
for epoch in range(num_epochs):
    model.train()
    running_loss=0.0

    for images,labels in train_loader:
        images,labels=images.to(device),labels.to(device)

        optimizer.zero_grad()
        outputs=model(images)
        loss=loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {running_loss/len(train_loader):.4f} | "
          f"Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.2f}%")


Epoch [1/11] | Train Loss: 0.2549 | Val Loss: 0.0804 | Val Acc: 97.43%
Epoch [2/11] | Train Loss: 0.0634 | Val Loss: 0.0503 | Val Acc: 98.37%
Epoch [3/11] | Train Loss: 0.0442 | Val Loss: 0.0419 | Val Acc: 98.71%
Epoch [4/11] | Train Loss: 0.0326 | Val Loss: 0.0414 | Val Acc: 98.63%
Epoch [5/11] | Train Loss: 0.0271 | Val Loss: 0.0395 | Val Acc: 98.67%
Epoch [6/11] | Train Loss: 0.0203 | Val Loss: 0.0420 | Val Acc: 98.69%
Epoch [7/11] | Train Loss: 0.0152 | Val Loss: 0.0689 | Val Acc: 98.04%
Epoch [8/11] | Train Loss: 0.0148 | Val Loss: 0.0513 | Val Acc: 98.40%
Epoch [9/11] | Train Loss: 0.0116 | Val Loss: 0.0466 | Val Acc: 98.70%
Epoch [10/11] | Train Loss: 0.0105 | Val Loss: 0.0457 | Val Acc: 98.81%
Epoch [11/11] | Train Loss: 0.0062 | Val Loss: 0.0474 | Val Acc: 98.79%
